# Case analysis:

Retail store, which has both physical and online stores, has $1000 left in marketing budget which won't roll over to the next year meaning that it has to be spent efficiently until the end of the year. The goal is to use this money to try and convert some physical store customers to the online store. Specifically, the intent is to mail some of these customers coupons for them to use online. Additionally, it has to be done without stealing physical store's customers.

I, as a data analyst, have to take several steps to complete this task.

(c) Miradiz Rakhmatov

# Data exploration:

In [1]:
import pandas as pd
import datetime as dt

In [2]:
retail = pd.read_csv('retail.txt', parse_dates=["trans_date"]) ## converts dates into datetime datatype

In [3]:
retail

,customer_id,trans_date,tran_amount
0,FM5295,2017-11-11,35
1,FM4768,2019-12-15,39
2,FM2122,2017-11-26,52
3,FM1217,2016-08-16,99
4,FM1850,2018-08-20,78
...,...,...,...
124995,FM8433,2016-03-26,64
124996,FM7232,2019-05-19,38
124997,FM8731,2019-08-28,42
124998,FM8133,2018-09-14,13


In [4]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


As you can see above, there are no null values in the dataset. Also, each column has the right corresponding data type.

# Step 1: Find churned customers (not so active customers)

Since the money is limited, it has to be used to send coupons for those customer that haven't made a purchase for quite some time. Although it may not be the perfect approach but I'm going to label customers that haven't purchased anything in over two months as churned. 


In [5]:
## Find the most recent transaction of each customer (grouping by customers)
## Find the most recent transaction of all transactions 

grouped_retail = pd.DataFrame(retail.groupby('customer_id')['trans_date'].max())
print("The most recent transaction date:", grouped_retail['trans_date'].max())
grouped_retail 

The most recent transaction date: 2019-12-16 00:00:00


,trans_date
customer_id,
FM1112,2019-10-14
FM1113,2019-11-09
FM1114,2019-11-12
FM1115,2019-12-05
FM1116,2019-05-25
...,...
FM8996,2019-09-09
FM8997,2019-03-28
FM8998,2019-09-22


In [6]:
## Thus 2019-10-16 would be two months from the last transaction at the store
## Let's find the customers that haven't purchased anything since October 16, 2019

grouped_retail['churned'] = grouped_retail['trans_date'].map(lambda x: 1 if x < pd.Timestamp(2019,10,16) else 0)
grouped_retail

,trans_date,churned
customer_id,,
FM1112,2019-10-14,1
FM1113,2019-11-09,0
FM1114,2019-11-12,0
FM1115,2019-12-05,0
FM1116,2019-05-25,1
...,...,...
FM8996,2019-09-09,1
FM8997,2019-03-28,1
FM8998,2019-09-22,1


In [7]:
grouped_retail['churned'].value_counts()

0    3671
1    3218
Name: churned, dtype: int64

As you can see about there are 3218 customers that haven't purchased anything in over three months. Dividing $1000 to all of them as a coupon wouldn't make any sense. Thus I have to find the best churned customers. 

# Step 2: Find the best customers among chunred customers

To classify customers I will use weighted sum model which has the following formula:
(Number of purchases/2) + (Amount Spent/2)

As per formula, we need total amount of purchases and total amount spent by each customer.

In [8]:
## Let's move sum of trans_amount from retail DataFrame to grouped_retail DataFrame by corresponding indexes (IDs)

grouped_retail['total_spent'] = retail.groupby('customer_id')['tran_amount'].sum()
grouped_retail

,trans_date,churned,total_spent
customer_id,,,
FM1112,2019-10-14,1,1012
FM1113,2019-11-09,0,1490
FM1114,2019-11-12,0,1432
FM1115,2019-12-05,0,1659
FM1116,2019-05-25,1,857
...,...,...,...
FM8996,2019-09-09,1,582
FM8997,2019-03-28,1,543
FM8998,2019-09-22,1,624


In [9]:
## Let's count how many transaction were made by each customer 

grouped_retail['number_of_trans'] = retail.groupby('customer_id')['tran_amount'].size()
grouped_retail

,trans_date,churned,total_spent,number_of_trans
customer_id,,,,
FM1112,2019-10-14,1,1012,15
FM1113,2019-11-09,0,1490,20
FM1114,2019-11-12,0,1432,19
FM1115,2019-12-05,0,1659,22
FM1116,2019-05-25,1,857,13
...,...,...,...,...
FM8996,2019-09-09,1,582,13
FM8997,2019-03-28,1,543,14
FM8998,2019-09-22,1,624,13


In [10]:
## Since churned customers are determined, trans_date column is no longer needed

grouped_retail.drop(columns='trans_date', inplace=True)

In [11]:
grouped_retail

,churned,total_spent,number_of_trans
customer_id,,,
FM1112,1,1012,15
FM1113,0,1490,20
FM1114,0,1432,19
FM1115,0,1659,22
FM1116,1,857,13
...,...,...,...
FM8996,1,582,13
FM8997,1,543,14
FM8998,1,624,13


As planned in the beginning of the step 2, I found total amount spent and number of transactions made by each customer. Let's reacll that these two columns are needed for weighted sum model to determine the best customers.

# Step 3: Weighted sum model

As you can see values in total_spent and number_of_trans are not relative to each other. To fix this, I will use min-max feature scaling which has the following formula: 

     (x - min(x)) / (max(x) - min(x)) where x is the column we want to scale.

In [12]:
grouped_retail['scaled_amount'] = (grouped_retail['total_spent'] - grouped_retail['total_spent'].min())/(grouped_retail['total_spent'].max()-grouped_retail['total_spent'].min())
grouped_retail['scaled_trans'] = (grouped_retail['number_of_trans'] - grouped_retail['number_of_trans'].min())/(grouped_retail['number_of_trans'].max()-grouped_retail['number_of_trans'].min())

Now that scaled amount and scaled transaction are relative to each other (apples-apples comparison), I can use weighted sum model formula to determine the best customers by assigning them as a score. 

In [13]:
## Calculating weighted sum model 
grouped_retail['score'] = ((grouped_retail['scaled_trans']/2) + (grouped_retail['scaled_amount']/2))*100

## Sort by the score in desending order 
grouped_retail.sort_values('score', inplace=True, ascending=False)
grouped_retail

,churned,total_spent,number_of_trans,scaled_amount,scaled_trans,score
customer_id,,,,,,
FM4424,0,2933,39,1.000000,1.000000,100.000000
FM4320,1,2647,38,0.897270,0.971429,93.434934
FM3799,1,2513,36,0.849138,0.914286,88.171182
FM5109,0,2506,35,0.846624,0.885714,86.616892
FM3805,1,2453,35,0.827586,0.885714,85.665025
...,...,...,...,...,...,...
FM7716,1,221,4,0.025862,0.000000,1.293103
FM7224,1,191,4,0.015086,0.000000,0.754310
FM8504,0,190,4,0.014727,0.000000,0.736351


# Step 4: Determine coupon value
Now that best customers are determined, I have to set the value for coupons. It will essentially give me the number of customer that will receive the coupons. 

$1000 / coupon value = number of customers with coupons

In [14]:
## Lets find the avarage purchase value from retail DataFrame

avg_pur_value = retail['tran_amount'].mean()
avg_pur_value


64.991912

In [15]:
## 30% of avarage purchase value would be the right bet 

coupon_value = avg_pur_value/100*30
coupon_value

19.4975736

In [16]:
## Let's round it up to $20. $1000/20=50 which means that 50 BEST CHURNED customers will recieve $20 worth coupon.

top_50_churned = grouped_retail[grouped_retail['churned'] == 1].head(50)
top_50_churned

,churned,total_spent,number_of_trans,scaled_amount,scaled_trans,score
customer_id,,,,,,
FM4320,1,2647,38,0.897270,0.971429,93.434934
FM3799,1,2513,36,0.849138,0.914286,88.171182
FM3805,1,2453,35,0.827586,0.885714,85.665025
FM5752,1,2612,33,0.884698,0.828571,85.663485
FM4074,1,2462,34,0.830819,0.857143,84.398091
FM1215,1,2362,35,0.794899,0.885714,84.030686
FM2620,1,2360,35,0.794181,0.885714,83.994766
FM1580,1,2329,33,0.783046,0.828571,80.580870
FM2951,1,2382,32,0.802083,0.800000,80.104167


# Conclusion:

top_50_churned dataset gives the marketing department a clear picture in:
* how much value each coupon should have - $20 each customer
* how many customers will recieve the coupon - 50 best customers in churned
* and most importantly which customers will recieve the coupons

# THE END